In [1]:
import requests
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [2]:
# Base API URL
BASE_URL = "https://fantasy.premierleague.com/api"

# Helper function to fetch JSON data
def fetch_data(url):
    return requests.get(url).json()

# Fetch data for the 2024-2025 season
bootstrap_static_2025 = fetch_data(f"{BASE_URL}/bootstrap-static/")
players_2025 = pd.DataFrame(bootstrap_static_2025['elements'])
teams_2025 = pd.DataFrame(bootstrap_static_2025['teams'])
positions_2025 = pd.DataFrame(bootstrap_static_2025['element_types'])

# Map team and position IDs to readable names
team_map_2025 = teams_2025.set_index('id')['name'].to_dict()
position_map_2025 = positions_2025.set_index('id')['singular_name_short'].to_dict()

# Add readable fields for players
players_2025['team'] = players_2025['team'].map(team_map_2025)
players_2025['position'] = players_2025['element_type'].map(position_map_2025)
players_2025['full_name'] = players_2025['first_name'] + " " + players_2025['second_name']

# Fetch current season (2024-2025) performance data
gameweek_data_2025 = []
for player_id in players_2025['id']:
    player_data = fetch_data(f"{BASE_URL}/element-summary/{player_id}/")
    history = player_data.get('history', [])  # Use 'history' for current season data

    if history:
        # Calculate total points, matches played, and other metrics from current gameweek data
        total_points = sum(week['total_points'] for week in history)
        minutes_played = sum(week['minutes'] for week in history)
        matches_played = minutes_played // 90  # Estimate games played
        
        gameweek_data_2025.append({
            'id': player_id,
            'full_name': players_2025.loc[players_2025['id'] == player_id, 'full_name'].values[0],
            'team': players_2025.loc[players_2025['id'] == player_id, 'team'].values[0],
            'position': players_2025.loc[players_2025['id'] == player_id, 'position'].values[0],
            'total_points': total_points,
            'matches_played': matches_played,
            'value': players_2025.loc[players_2025['id'] == player_id, 'now_cost'].values[0] / 10  # Value in millions
        })

# Convert to DataFrame
df_2025 = pd.DataFrame(gameweek_data_2025)

# Calculate PPM (Points Per Million) and Points Per Match
df_2025['points_per_match'] = df_2025['total_points'] / df_2025['matches_played']
df_2025['ppm'] = df_2025['points_per_match'] / df_2025['value']

df_2025.to_csv('FPL 2024-2024 Player Performance.csv', index=False)
df_2025.head()

,id,full_name,team,position,total_points,matches_played,value,points_per_match,ppm
0,1,Fábio Ferreira Vieira,Arsenal,MID,0,0,5.4,NaN,NaN
1,2,Gabriel Fernando de Jesus,Arsenal,FWD,9,2,6.8,4.500000,0.661765
2,3,Gabriel dos Santos Magalhães,Arsenal,DEF,60,12,6.2,5.000000,0.806452
3,4,Kai Havertz,Arsenal,FWD,59,13,7.9,4.538462,0.574489
4,5,Karl Hein,Arsenal,GKP,0,0,4.0,NaN,NaN


In [3]:
# Calculate the total number of matches played by any team so far
# Assuming the maximum matches played by any player is the total matches played so far
total_games_so_far = df_2025['matches_played'].max()  # Use the maximum matches played

# Filter the dataframe for players who played at least 60% of the games
filtered_df = df_2025[df_2025['matches_played'] >= 0.6 * total_games_so_far]

# Extract the top 10 players for each position based on points per match
top_10_per_position = (
    filtered_df.sort_values(by='points_per_match', ascending=False)
    .groupby('position')
    .head(10)
)

# Keep only the relevant columns and order the output
top_10_per_position = top_10_per_position[['full_name', 'position', 'team', 'points_per_match', 'matches_played', 'value']]
top_10_per_position = top_10_per_position.sort_values(by=['position', 'points_per_match'], ascending=[True, False])
top_10_per_position

,full_name,position,team,points_per_match,matches_played,value
385,Trent Alexander-Arnold,DEF,Liverpool,6.100000,10,7.0
279,Ashley Young,DEF,Everton,5.181818,11,4.6
2,Gabriel dos Santos Magalhães,DEF,Arsenal,5.000000,12,6.2
399,Ibrahima Konaté,DEF,Liverpool,4.600000,10,5.2
411,Virgil van Dijk,DEF,Liverpool,4.571429,14,6.4
427,Joško Gvardiol,DEF,Man City,4.384615,13,6.1
5,Jurriën Timber,DEF,Arsenal,4.272727,11,5.6
408,Andrew Robertson,DEF,Liverpool,4.272727,11,5.9
14,William Saliba,DEF,Arsenal,4.230769,13,6.2
515,Ola Aina,DEF,Nott'm Forest,4.142857,14,4.9


In [4]:
# Create the optimization problem
prob = LpProblem("FPL_Team_Selection", LpMaximize)

# Create a binary variable for each player (1 if selected, 0 otherwise)
player_vars = {i: LpVariable(f"player_{i}", cat='Binary') for i in top_10_per_position.index}

# Objective function: Maximize total points per match (PPM)
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'points_per_match'] for i in top_10_per_position.index)

# Constraints

# Total budget <= 100
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'value'] for i in top_10_per_position.index) <= 100

# Total players = 15
prob += lpSum(player_vars[i] for i in top_10_per_position.index) == 15

# Position constraints
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'GKP') == 2
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'DEF') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'MID') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'FWD') == 3

# Solve the problem
prob.solve()

# Extract the selected players
selected_players = top_10_per_position.loc[[i for i in top_10_per_position.index if player_vars[i].value() == 1]]

# Order by position and points per match
selected_players = selected_players.sort_values(by=['position', 'points_per_match'], ascending=[True, False])

selected_players

,full_name,position,team,points_per_match,matches_played,value
385,Trent Alexander-Arnold,DEF,Liverpool,6.100000,10,7.0
279,Ashley Young,DEF,Everton,5.181818,11,4.6
2,Gabriel dos Santos Magalhães,DEF,Arsenal,5.000000,12,6.2
399,Ibrahima Konaté,DEF,Liverpool,4.600000,10,5.2
515,Ola Aina,DEF,Nott'm Forest,4.142857,14,4.9
127,Yoane Wissa,FWD,Brentford,8.111111,9,6.2
539,Chris Wood,FWD,Nott'm Forest,7.333333,12,6.5
304,Raúl Jiménez,FWD,Fulham,6.777778,9,5.5
466,André Onana,GKP,Man Utd,4.333333,15,5.2
276,Jordan Pickford,GKP,Everton,4.142857,14,4.9


In [5]:
selected_players['points_per_match'].sum()

97.65593295593295

In [6]:
selected_players['value'].sum()

99.99999999999999